In [3]:
import numpy as np
import scipy.io
import os
import pandas as pd
import math
from pathlib import Path
from mayavi import mlab
import copy

Set the root directory where you have your .mat files

## matlab_loader

This makes a data loader class for matlab files and also passes a filter to remove noise 

In [4]:
class matlab_loader:
    """ Loads .mat files from a direcotory 
            
            root_dir (str): path to the directory that containes files
            mat_file (str): name of file 
     
         returns:
             self.array (np.array)
             self.mat_file (Path object)
    """
    def __init__(self, root_dir, mat_file):  
        
        root_dir = Path(root_dir)
        mat_file = Path(mat_file)
        
        mat_obj = scipy.io.loadmat(root_dir/mat_file)
        obj= mat_obj.keys()
        obj = list(obj)
        array = mat_obj[obj[-1]]
        
        self.array = array
        self.mat_file = root_dir/mat_file
        
    def __repr__(self):
        return f'{self.mat_file}'

In [5]:
def xyz(arr, filter_level = 0.3):
    """Convert 3D voxel arry to xyz coordiates.
    
            arr (np.array): 3D voxel array    
            filter_level (int/float): sets the threshold level for what is considered a voxel 
            
            returns: 
                np.array (n x 3)
    """
    
    # Everything above filter level is converted to 1
    arr = np.where(arr < filter_level, 0, 1)
    
    x, y, z = np.where(arr == 1)
    
    # converts the xyz so z is is *up* 
    x -= arr.shape[1]
    y -= arr.shape[0]
    x *= -1
    y *= -1
    xyz = np.array([x, y, z]).T
    return xyz

## PCA

In [6]:
def pca(xyz):
    """PCA on a xyz points array
    
            xyz(np.array): n x 3 array of xyz coordinates
            
            returns:
                mean_x 
                mean_y
                mean_z
                eig_val (1 x 3)
                eig_vec (1 x 3)
    """
    
    #covaraince of xyz points
    cov = np.cov(xyz.T)
    
    #eiganvalues and vectors
    (eig_val, eig_vec) = np.linalg.eig(cov)

    mean_x, mean_y, mean_z = [np.mean(xyz[:,0]),np.mean(xyz[:,1]),np.mean(xyz[:,2])]

    return mean_x, mean_y, mean_z, eig_val, eig_vec

## Classes:

### Bone

In [7]:
class bone:
        def __init__(self, array, filter_level=0.3):
          
            if array is None:
                pass
            
            else:
                self.array = array
                self.filter_level = filter_level
                self.xyz = xyz(array, filter_level)

                mean_x, mean_y, mean_z, eig_val, eig_vec = pca(self.xyz) 

                self.vec = eig_vec
                self.PC1 = eig_vec[:,0]
                self.PC2 = eig_vec[:,1]
                self.PC3 = eig_vec[:,2]   

                self.mean = (mean_x, mean_y, mean_z)

### Foot bone

### Indivual bone classes

In [8]:
class foot_bone(bone):
    def __init__(self, 
                 phantom = None,
                 ang3 = None,
                 ang6 = None,
                 ang10 = None, 
                 ang15 = None):
        
        self.phantom = phantom
        self.ang3 = ang3
        self.ang6 = ang6
        self.ang10 = ang10
        self.ang15 = ang15
        
    def __repr__(self):
        return self.name

In [9]:
# NEEDS TO BE REFACTORED
class phantom():
    def __init__(self, f1 = None ,f2 = None):
        self.f1 = bone(f1)
        self.f2 = bone(f2)

    def __repr__(self):
        return 'phantom '

class ang3():
    def __init__(self, f1 = None ,f2 = None):
        self.f1 = bone(f1)
        self.f2 = bone(f2)
        
    def __repr__(self):
        return 'ang3 '

class ang6():
    def __init__(self, f1 = None ,f2 = None):
        self.f1 = bone(f1)
        self.f2 = bone(f2)
        
    def __repr__(self):
            return 'ang6 '
        
class ang10():
    def __init__(self, f1 = None ,f2 = None):
        self.f1 = bone(f1)
        self.f2 = bone(f2)
        
    def __repr__(self):
        return 'ang10 '
        
class ang15():
    def __init__(self, f1 = None ,f2 = None):
        self.f1 = bone(f1)
        self.f2 = bone(f2)
        
    def __repr__(self):
        return 'ang15'

## Ploting:

## bone_plot

In [10]:
def bone_plot(*args, user_colours = None, plot_PCA = True, plot_inv = True):
    """plots voxel array; can take n bones and plot PCA vectors"""

    # Sorting out colours
    colour_dict = {'yellow':(0.9,0.9,0),
                   'pastel_blue':(0.7,1,1),
                   'purple':(0.6,0,0.5),
                   'orange':(0.8,0.3,0),
                   'dark_blue':(0,0.3,0.7),}
    
    if user_colours is None:
        user_colours = colour_dict
    
    plot_colours = []
    
    for col in user_colours:
        x = colour_dict.get(col)
        plot_colours.append(x)
        
    
    for n, bone in enumerate(args):

        mlab.points3d(bone.xyz[:,0],
                             bone.xyz[:,1],
                             bone.xyz[:,2],
                     mode="cube",
                     color= plot_colours[n],
                     scale_factor=1)
        
        x,y,z = bone.mean

        #plot princible vectors
        u0,v0,w0 = bone.PC1 * 100
        u0_inv,v0_inv,w0_inv = bone.PC1 * 100 * -1

        u1,v1,w1 = bone.PC2 * 100
        u1_inv,v1_inv,w1_inv = bone.PC2 * 100 * -1

        u2,v2,w2 = bone.PC3 * 100
        u2_inv,v2_inv,w2_inv = bone.PC3 * 100 * -1

        print(f"{n}th bone PCA vectors: \n {bone.vec} \n ")
        
        
        if plot_PCA is True:
            mlab.quiver3d(x,y,z,u0,v0,w0,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
            mlab.quiver3d(x,y,z,u1,v1,w1,
                                 line_width =6,
                                 scale_factor= 0.5,
                                 color= (0,1,0))
            mlab.quiver3d(x,y,z,u2,v2,w2,
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))


        #ploting the inverse of eigen vectors
        if plot_inv is True:
             mlab.quiver3d(x,y,z,u0_inv,v0_inv,w0_inv,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
             mlab.quiver3d(x,y,z,u1_inv,v1_inv,w1_inv,
                                 line_width =6,
                                 scale_factor=0.5,
                                 color= (0,1,0))
             mlab.quiver3d(x,y,z,u2_inv,v2_inv,w2_inv,
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))

    return mlab.show()

In [11]:
# Option user colours
# user_colours=['yellow','purple']

In [12]:
# bone_plot(phantom_tibia_f1,phantom_tibia_f2)

## Angels

In [13]:
def mag(v):
    """Finds magnitude of vector
        v (np.array): vector
    
        returns: magnitude of vector (numeric)
    """
    return math.sqrt(v @ v)

#angel of vector
def angle(v1, v2):
    """Finds angel between vectors"""
    try:
        x = math.acos(v1 @ v2 / (mag(v1) * mag(v2)))
    
    except:
        x = 0
        
    return x

In [14]:
root_dir = Path('C:\\Users\luke\OneDrive - University College London\Marta\data')

In [15]:
root_dir

WindowsPath('C:/Users/luke/OneDrive - University College London/Marta/data')

In [16]:
%%time
# load data
tibia_phant_f2 = matlab_loader(root_dir, mat_file = 'phantom/phantom_talus_f2.mat' )
tibia_phant_f1 = matlab_loader(root_dir, mat_file = 'phantom/phantom_talus_f1.mat')
tibia_3_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/3 angles/talus_f2.mat')
tibia_6_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/6 angles/talus_f2.mat')
tibia_10_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/10 angles/talus_f2.mat')
tibia_15_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/15 angles/talus_f2.mat')

Wall time: 15.4 s


In [17]:
#root_dir = Path('C:\\Users\luke\Downloads\low_res')

In [18]:
# tibia_phant_f2 = matlab_loader(root_dir, mat_file = 'talus_f2.mat')
# tibia_phant_f1 = matlab_loader(root_dir, mat_file = 'talus_f1.mat')
# tibia_3_f2 = matlab_loader(root_dir, mat_file = 'fista recons/3 angles/talus_f2.mat')
# tibia_6_f2 = matlab_loader(root_dir, mat_file = 'fista recons/6 angles/talus_f2.mat')

In [19]:
%%time
tibia = foot_bone(
    phantom(f1=tibia_phant_f1.array, 
            f2=tibia_phant_f2.array),
    ang3(f2 = tibia_3_f2.array),
    ang6(f2 = tibia_6_f2.array),
    ang10(f2 = tibia_10_f2.array),
    ang15(f2 = tibia_15_f2.array)
)

Wall time: 2.1 s


In [20]:
# Refactor so you can define name in class

In [21]:
tibia.name = 'tibia'

In [22]:
# have __iter__ so you can loop through PCAs
def df_angles(bone, ang3 = None, ang6 = None, ang10 = None, ang15 = None, ALL = None):
    
    df = pd.DataFrame()
    
    #Phant f1 vs Phant f2 
    df.loc[f'{bone} phantom f1: PC1' ,f'{bone} phantom f2: PC1'] = angle(bone.phantom.f1.PC1,bone.phantom.f2.PC1)
    df.loc[f'{bone} phantom f1: PC2' ,f'{bone} phantom f2: PC2'] = angle(bone.phantom.f1.PC2,bone.phantom.f2.PC2)
    df.loc[f'{bone} phantom f1: PC3' ,f'{bone} phantom f2: PC3'] = angle(bone.phantom.f1.PC3,bone.phantom.f2.PC3)
 
    if ang3 or ALL is True:
        #Phant f2 vs ang_3 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang3 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang3.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang3 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang3.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang3 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang3.f2.PC3)

    if ang6 or ALL is True:
        #Phant f2 vs ang_6 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang6 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang6.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang6 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang6.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang6 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang6.f2.PC3)
    
    if ang10 or ALL is True:
        #Phant f2 vs ang_10 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang10 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang10.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang10 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang10.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang10 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang10.f2.PC3)
    
    if ang10 or ALL is True:
        #Phant f2 vs ang_15 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang10 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang15.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang10 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang15.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang10 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang15.f2.PC3)
    
    return df

In [23]:
tibia.ang10.f2.PC1 , tibia.phantom.f2.PC1
#point excption of same

(array([-0.60124372,  0.59393072,  0.53455803]),
 array([-0.60124372,  0.59393072,  0.53455803]))

In [24]:
%%time
df_angles(tibia, ALL = True)

Wall time: 19.9 ms


,tibia phantom f2: PC1,tibia phantom f2: PC2,tibia phantom f2: PC3,tibia ang3 f2: PC1,tibia ang3 f2: PC2,tibia ang3 f2: PC3,tibia ang6 f2: PC1,tibia ang6 f2: PC2,tibia ang6 f2: PC3,tibia ang10 f2: PC1,tibia ang10 f2: PC2,tibia ang10 f2: PC3
tibia phantom f1: PC1,2.523024,NaN,NaN,2.521118,NaN,NaN,2.523521,NaN,NaN,2.523024,NaN,NaN
tibia phantom f1: PC2,NaN,2.576199,NaN,NaN,2.564794,NaN,NaN,2.576205,NaN,NaN,2.576199,NaN
tibia phantom f1: PC3,NaN,NaN,0.303911,NaN,NaN,0.296299,NaN,NaN,0.304239,NaN,NaN,0.303911


In [25]:
# bone_plot(tibia.phantom.f1)

# Rotation

## Step 1: Center the 2 means

displacement of PC1 and PC1_pos2

then rotate around that vector the angels of PC2 and PC3

In [26]:
tfm =  np.asarray(tibia.phantom.f1.mean) - np.asarray(tibia.phantom.f2.mean)

In [27]:
tibia.phantom.f1.xyz

array([[140, 148,  99],
       [140, 148, 100],
       [140, 147,  97],
       ...,
       [109, 155,  99],
       [109, 155, 100],
       [109, 154,  99]], dtype=int64)

In [28]:
tfm_tib_f1 = tibia.phantom.f1.xyz + tfm

In [29]:
tfm_tib_f1

array([[139.55373319, 148.13572273, 101.46922275],
       [139.55373319, 148.13572273, 102.46922275],
       [139.55373319, 147.13572273,  99.46922275],
       ...,
       [108.55373319, 155.13572273, 101.46922275],
       [108.55373319, 155.13572273, 102.46922275],
       [108.55373319, 154.13572273, 101.46922275]])

In [30]:
mlab.points3d(tfm_tib_f1[:,0],
              tfm_tib_f1[:,1],
              tfm_tib_f1[:,2], 
              mode="cube",color=(0,1,0))

mlab.points3d(tibia.phantom.f2.xyz[:,0],
              tibia.phantom.f2.xyz[:,1],
              tibia.phantom.f2.xyz[:,2], 
              mode="cube", scale_factor=1,color= (1,0,0))


mlab.show()

## Change coordinate orgigen (bone)

Orgigen is now mean of the bone

In [31]:
tfm_tib_f1

array([[139.55373319, 148.13572273, 101.46922275],
       [139.55373319, 148.13572273, 102.46922275],
       [139.55373319, 147.13572273,  99.46922275],
       ...,
       [108.55373319, 155.13572273, 101.46922275],
       [108.55373319, 155.13572273, 102.46922275],
       [108.55373319, 154.13572273, 101.46922275]])

In [32]:
#changing bone matrix coords f2
tfm_tib_f1[:,0] -= tibia.phantom.f1.mean[0]
tfm_tib_f1[:,1] -= tibia.phantom.f1.mean[1]
tfm_tib_f1[:,2] -= tibia.phantom.f1.mean[2]

In [33]:
tib_f2 = tibia.phantom.f2.xyz

In [34]:
tib_f2 = tib_f2.astype(np.float64)

In [35]:
tib_f2[:,0] -= tibia.phantom.f1.mean[0]
tib_f2[:,1] -= tibia.phantom.f1.mean[1]
tib_f2[:,2] -= tibia.phantom.f1.mean[2]

In [36]:
mlab.points3d(tfm_tib_f1[:,0],
              tfm_tib_f1[:,1],
              tfm_tib_f1[:,2], 
              mode="cube",color=(0,1,0))

mlab.points3d(tib_f2[:,0],
              tib_f2[:,1],
              tib_f2[:,2], 
              mode="cube", scale_factor=1,color= (1,0,0))
mlab.show()

## Add PCAs

In [37]:
x1,y1,z1 = tibia.phantom.f1.PC1
x2,y2,z2 = tibia.phantom.f1.PC2
x3,y3,z3 = tibia.phantom.f1.PC3

In [38]:
f2_x1,f2_y1,f2_z1 = tibia.phantom.f2.PC1
f2_x2,f2_y2,f2_z2 = tibia.phantom.f2.PC2
f2_x3,f2_y3,f2_z3 = tibia.phantom.f2.PC3

In [39]:
x1,y1,z1

(0.23452909433617208, -0.9503885821121647, -0.20434687886210598)

In [40]:
x2,y2,z2

(-0.9484961296519098, -0.26976960470439565, 0.16607062477446716)

In [41]:
# mlab.points3d(tfm_tib_f1[:,0],
#               tfm_tib_f1[:,1],
#               tfm_tib_f1[:,2], 
#               mode="cube",color=(0,1,0))

# mlab.points3d(tib_f2[:,0],
#               tib_f2[:,1],
#               tib_f2[:,2], 
#               mode="cube", scale_factor=1,color= (1,0,0))
#f1
mlab.quiver3d(0,0,0,x1,y1,z1, line_width =6, scale_factor= 50, color= (1,0,0))
mlab.quiver3d(0,0,0,x2,y2,z2, line_width =6, scale_factor= 50, color= (0,1,0))
mlab.quiver3d(0,0,0,x3,y3,z3, line_width =6, scale_factor= 50, color= (0,0,1))

#f2 PCA 
mlab.quiver3d(0,0,0,-f2_x1,-f2_y1,-f2_z1, line_width =6, scale_factor= 100, color= (1,0,0))
mlab.quiver3d(0,0,0,f2_x2,f2_y2,f2_z2, line_width =6, scale_factor= 100, color= (0,0,1))
mlab.quiver3d(0,0,0,f2_x3,f2_y3,f2_z3, line_width =6, scale_factor= 100, color= (0,1,0))

mlab.show()

## Quatertions

In [110]:
import quaternion

In [47]:
from pyquaternion import Quaternion

In [ ]:
def rotation(bone_f1, bone_f2):
    
    # PC1 
    # angle
    ang = angle(bone_f1, bone_f2)
    
    # cross product
    #x,y,z = np.cross(bone_f1, bone_f2)
    # Quaternion around crossproduct
    
    r = Quaternion(axis = np.cross(bone_f1, bone_f2), angle = ang)
    
    for pc in bone_f1
        bone_f1.pc = r.rotate(bone_f1)

        x = np.apply_along_axis(lambda x: r.rotate(x), 1, tfm_tib_f1)

## Rotaion around PCs

Have to invtert the vectors if they are going the wrong way or it won't work

In [78]:
#pc1_inv = tibia.phantom.f2.PC1*-1

...pc1

In [79]:
#ang =  angle(tibia.phantom.f1.PC1 ,pc1_inv)

In [113]:
ang =  angle(tibia.phantom.f1.PC1 ,tibia.phantom.f2.PC1)

In [114]:
ang

2.5230241705271723

In [115]:
#x,y,z = np.cross(tibia.phantom.f1.PC1,pc1_inv)

In [116]:
x,y,z = np.cross(tibia.phantom.f1.PC1,tibia.phantom.f2.PC1)

In [117]:
r1 = Quaternion(axis = [x,y,z], angle = ang)

In [118]:
r1

Quaternion(0.3043769175416624, -0.6351827892880586, -0.004118466287388444, -0.7098454440901326)

In [ ]:
#mag_sq = np.dot(axis, axis)

In [ ]:
#axis = axis / sqrt(mag_sq)

In [123]:
#r  = np.quaternion(math.cos(ang/2),
                   x*math.sin(ang/2), y*math.sin(ang/2), z*math.sin(ang/2))

In [124]:
r

quaternion(0.304376917541662, -0.368323116777777, -0.00238817292423076, -0.411617711951595)

In [86]:
pc1 = r1.rotate(tibia.phantom.f1.PC1)
pc2 = r1.rotate(tibia.phantom.f1.PC2)
pc3 = r1.rotate(tibia.phantom.f1.PC3)

In [87]:
x = np.apply_along_axis(lambda x: r1.rotate(x), 1, tfm_tib_f1)

...pc2

In [88]:
#pc2_inv = tibia.phantom.f2.PC2*-1

In [89]:
#ang = (angle(pc2_inv ,tibia.phantom.f1.PC2))

In [90]:
ang = (angle(tibia.phantom.f2.PC2 ,pc2))

In [91]:
ang

0.9365642423615254

In [92]:
x,y,z= np.cross(pc2,tibia.phantom.f2.PC2)

In [93]:
r2 = Quaternion(axis = [x,y,z], angle = ang)

In [94]:
# r2  = np.quaternion(math.cos(ang)/2, 
#                     x[0]*math.sin(ang/2), y[0]*math.sin(ang/2), z[0]*math.sin(ang/2))

In [95]:
pc1 = r2.rotate(pc1)
pc2 = r2.rotate(pc2)
pc3 = r2.rotate(pc3)

In [96]:
x = np.apply_along_axis(lambda x: r2.rotate(x), 1, tfm_tib_f1)

...pc3

In [97]:
#pc3_inv = tibia.phantom.f2.PC3*-1

In [98]:
#ang = (angle(pc3_inv ,tibia.phantom.f1.PC3))

In [99]:
ang =  angle(tibia.phantom.f2.PC3 ,pc3)

In [100]:
ang

0

In [101]:
x,y,z= np.cross(pc3, tibia.phantom.f2.PC3)

In [102]:
r3 = Quaternion(axis = [x,y,z], angle = ang)

In [103]:
# r3  = np.quaternion(math.cos(ang)/2, 
#                     x[0]*math.sin(ang/2), y[0]*math.sin(ang/2), z[0]*math.sin(ang/2))

Combind Quaterion rotataions

In [104]:
x = np.apply_along_axis(lambda x: r3.rotate(x), 1, tfm_tib_f1)

In [105]:
pc1 = r3.rotate(pc1)
pc2 = r3.rotate(pc2)
pc3 = r3.rotate(pc3)

In [106]:
# roatated 
mlab.points3d(x[:,0],
              x[:,1],
              x[:,2], 
              mode="cube", color=(0,0.7,0))

#f2
mlab.points3d(tib_f2[:,0],
              tib_f2[:,1],
              tib_f2[:,2],
              mode="cube", color=(1,0,0))

#f1 PCA 
#mlab.quiver3d(0,0,0,x1,y1,z1, line_width =6, scale_factor= 50, color= (0.6,0,0))
#mlab.quiver3d(0,0,0,x2,y2,z2, line_width =6, scale_factor= 50, color= (0,1,0))
#mlab.quiver3d(0,0,0,x3,y3,z3, line_width =6, scale_factor= 50, color= (0,0,1))

#f2 PCA 
mlab.quiver3d(0,0,0, f2_x1, f2_y1, f2_z1, line_width =6, scale_factor= 100, color= (1,0,0))
mlab.quiver3d(0,0,0,f2_x2,f2_y2,f2_z2, line_width =6, scale_factor= 100, color= (0,0,1))
mlab.quiver3d(0,0,0,f2_x3,f2_y3,f2_z3, line_width =6, scale_factor= 100, color= (0,1,0))

#w/ rotion
mlab.quiver3d(0,0,0,pc1[0],pc1[1],pc1[2], line_width =6, scale_factor= 100, color= (0.6,0,0))
mlab.quiver3d(0,0,0,pc2[0],pc2[1],pc2[2], line_width =6, scale_factor= 100, color= (0,0,1))
mlab.quiver3d(0,0,0,pc3[0],pc3[1],pc3[2], line_width =6, scale_factor= 100, color= (0,1,0))

#mlab.quiver3d(0,0,0,nx,ny,nz, line_width =6, scale_factor= 100, color= (0,0,0))


mlab.show()

In [227]:
tibia.phantom.f1.xyz.shape

(21730, 3)

## test

In [ ]:
p0 = tibia.phantom.f2.PC1; p0

In [ ]:
p1 = tibia.phantom.f1.PC1

In [ ]:
def norm(vec):
    x,y,z = vec
    
    n = 1/(x+y+z)
    
    return np.array([(n*x),(n*y),(n*z)])

In [ ]:
mag(p0)

In [ ]:
p0 = p0/mag(p0)
p1 = p1/mag(p1)

In [ ]:
p0

In [ ]:
p0[0]+ p0[1]+p0[2] , p1[0]+ p1[1]+p1[2]

In [ ]:
#p0 = np.array([0.5,0.5,0]) 

In [ ]:
#from sklearn.preprocessing import normalize

In [ ]:
#p1 = np.array([0,0,1])

In [ ]:
xc, yc, zc = np.cross(p0, p1)

In [ ]:
#vc = norm(np.array([xc, yc, zc]))

In [ ]:
xc + yc + zc

In [ ]:
ang =  angle(p1,p0); ang

In [ ]:
angle(p1,p0) == angle(p0,p1)

In [ ]:
r  = np.quaternion((math.cos(ang/2)), 
                    xc*math.sin(ang/2), yc*math.sin(ang/2), zc*math.sin(ang/2))

In [ ]:
(math.cos(ang/2)**2)+  (xc*math.sin(ang/2))**2 + (yc*math.sin(ang/2))**2 +(zc*math.sin(ang/2))**2

In [ ]:
xr = quaternion.rotate_vectors(r, p0) ;xr

In [ ]:
# OG point
mlab.quiver3d(0,0,0,p0[0],p0[1],p0[2],scale_factor=1, line_width =6, color= (0.6,0,0))

# Target point
mlab.quiver3d(0,0,0,p1[0],p1[1],p1[2], line_width =6, color= (0.6,0,0))

# Rotated point
mlab.quiver3d(0,0,0,xr,yr,zr, line_width =6, color= (0,0.6,0))

mlab.points3d(x[:,0],
              x[:,1],
              x[:,2], 
              mode="cube", color=(0,0.7,0))

mlab.show()

In [ ]:
xr, yr, zr = q.rotate(p0)

In [ ]:
q = Quaternion(axis = [xc,yc,zc], angle = ang)

In [ ]:
q

In [ ]:
x = np.apply_along_axis(lambda x: q.rotate(x), 1, tibia.phantom.f1.xyz)

In [ ]:
x

In [236]:
mlab.points3d(tibia.phantom.f1.xyz[:,0],
                tibia.phantom.f1.xyz[:,1],
                tibia.phantom.f1.xyz[:,2], 
                mode="cube", color=(0,0.7,0))

#f2
mlab.points3d(tibia.phantom.f2.xyz[:,0],
               tibia.phantom.f2.xyz[:,1],
               tibia.phantom.f2.xyz[:,2],
               mode="cube", color=(0.6,0,0))

mlab.show()


In [234]:
tibia.phantom.f1.mean

(122.94362632305568, 154.3126553152324, 133.98301886792453)

In [235]:
tibia.phantom.f2.mean

(123.12932705621711, 154.04426290039694, 134.1327887011908)